In [194]:
# 리밸런싱 
# out-of-sample에서 리밸런싱 우선
# 매달 리밸런싱 진행
    # 리밸런싱은 해당 기간 최적화 돌려서 나온 비중을 갖고 '매달' 진행.
    # 1. (out of sample 기간) 2023.05.01에 투자를 시작할 때 과거 5년치 데이터로 최적화 문제 풀고, 다음 한달간 수익률 보기. 
    # 그리고 2023.06.01에 시작할 때는 그때까지 데이터로 최적화 풀고, 
    # 나온 가중치로 리밸런싱 하고 한달간 수익률 보기… 
    # 반복
# 2. look back 기간을 1년, 2년, 등 여러 기간으로 설정하고 돌려보기.

In [195]:
# !pip install pyPortfolioOpt
# !pip install cplex
# !pip install numpy
# !pip install pandas
# !pip install cvxpy

In [196]:
import numpy as np
import cvxpy as cp
import cplex
from tqdm import tqdm
from datetime import timedelta

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader.data as pdr
import yfinance as yf

from dateutil.relativedelta import relativedelta
from datetime import datetime

In [197]:
# GPT 종목 (GPT weight)
stocks_15 = {
    "AAPL": 0.084, "AMZN": 0.074, "NVDA": 0.064, "JPM": 0.054, "PG": 0.054,
    "PFE": 0.064, "JNJ": 0.064, "KO": 0.054, "XOM": 0.064, "NEE": 0.074,
    "GOOGL": 0.084, "MSFT": 0.084, "TSLA": 0.074, "NKE": 0.054, "BAC": 0.054
}

stocks_30 = {
    "MSFT": 0.067, "AMZN": 0.067, "NVDA": 0.067, "AAPL": 0.067, "GOOGL": 0.067,
    "ADBE": 0.067, "JNJ": 0.05, "PFE": 0.05, "MRK": 0.05, "ABT": 0.05, 
    "PG": 0.05, "KO": 0.05, "JPM": 0.05, "GS": 0.05, "CAT": 0.025, 
    "CVX": 0.025, "XOM": 0.025, "BA": 0.025, "TSLA": 0.025, "NEE": 0.025, 
    "NKE": 0.005, "VZ": 0.005, "CRM": 0.005, "UNH": 0.005, "WMT": 0.005, 
    "QCOM": 0.005, "BAC": 0.005, "V": 0.005, "MCD": 0.005, "INTC": 0.005
}

stocks_45 = {
    "AAPL": 0.05, "GOOGL": 0.04, "MSFT": 0.04, "NVDA": 0.03, "AMD": 0.03, 
    "ORCL": 0.02, "CRM": 0.02, "INTC": 0.01, "CSCO": 0.01, "JPM": 0.04, 
    "GS": 0.03, "BAC": 0.03, "MS": 0.02, "AXP": 0.02, "C": 0.01,
    "JNJ": 0.03, "UNH": 0.03, "PFE": 0.02, "ABBV": 0.02, "TSLA": 0.02, 
    "AMGN": 0.02, "GILD": 0.01, "PG": 0.03, "KO": 0.03, "NKE": 0.02, 
    "PEP": 0.02, "COST": 0.02, "WMT": 0.02, "TGT": 0.01, "XOM": 0.025, 
    "CVX": 0.025, "NEE": 0.02, "DUK": 0.01, "SO": 0.01, "SLB": 0.01,
    "MMM": 0.02, "CAT": 0.02, "HON": 0.02, "GE": 0.02, "ADP": 0.02,
    "AMZN": 0.02, "META": 0.02, "HD": 0.02, "VZ": 0.01, "MRK": 0.01
}

In [198]:
# 종목명 리스트
tickers_15 = list(stocks_15.keys())
tickers_30 = list(stocks_30.keys())
tickers_45 = list(stocks_45.keys())

## GPT weight

In [199]:
gpt_15_weights = stocks_15
gpt_30_weights = stocks_30
gpt_45_weights = stocks_45

## Equal weight

In [200]:
eq_15_weights = [1/15] * 15
eq_30_weights = [1/30] * 30
eq_45_weights = [1/45] * 45

eq_15 =dict(zip(tickers_15, eq_15_weights))
eq_30 =dict(zip(tickers_30, eq_30_weights))
eq_45 =dict(zip(tickers_45, eq_45_weights))

## GMV + EPO weight

In [201]:
gmv_epo_15_weights = [0.049605485006088634, 0.07849495425750663, 0.06333929068001071, 0.05700662407849795, 0.07650289062642221, 0.05900628444525451, 0.057323863755786, 0.0475050526473884, 0.06851785110258568, 0.0551095984517666, 0.0635746953436858, 0.08684669491598282, 0.06555603211482636, 0.08160761317658853, 0.090003069397609]
gmv_epo_30_weights = [0.016666779461997225, 0.028582382826203727, 0.03181481366190954, 0.04612454337672658, 0.03430208600932986, 0.02496457702197057, 0.034559689579773685, 0.03603933972602465, 0.0348256115466567, 0.02719368909866466, 0.01666666768426058, 0.03550281897656019, 0.03316982644195968, 0.02106295226563651, 0.019372147251594442, 0.01873074735009172, 0.06623366706426345, 0.016666690262465863, 0.03423317149469612, 0.02304327143120648, 0.031214508772612753, 0.045807643648171685, 0.027946448832753296, 0.049668167222889026, 0.049588148769328606, 0.024769224302769302, 0.016666667674594095, 0.05283625719167181, 0.05583979037764232, 0.045907670675591185]
gmv_epo_45_weights = [0.011111111975729409, 0.026646602204962858, 0.011111111299922065, 0.0309210193238956, 0.025007258123960922, 0.034659892275724705, 0.01111111146901072, 0.011111111747740477, 0.011111111516746397, 0.01779091912009698, 0.01984888218960698, 0.02522647335013622, 0.011180137115527101, 0.017489649124790285, 0.028191045792364738, 0.03631158995254371, 0.044444444090877805, 0.016302194182152667, 0.011111111266490585, 0.011111111595258412, 0.01111111125737019, 0.022827040798679366, 0.020587601244674014, 0.011111111503325005, 0.011111111556128327, 0.044444443639370106, 0.011111111461049875, 0.04444444402780661, 0.011111111368812045, 0.011111112285545484, 0.023035029376855205, 0.01260693000477375, 0.02165124524507929, 0.02165119066334294, 0.011286898608143595, 0.029528803525066108, 0.017506358987491956, 0.03972350417693582, 0.018956379525549516, 0.036275899230639834, 0.040688577981395774, 0.011896549529971394, 0.03665347882393091, 0.03968317677255412, 0.028087890687943526]

gmv_epo_15 = dict(zip(tickers_15, gmv_epo_15_weights))
gmv_epo_30 = dict(zip(tickers_30, gmv_epo_30_weights))
gmv_epo_45 = dict(zip(tickers_45, gmv_epo_45_weights))

## MAX Sharpe + EPO weight

In [202]:
msp_epo_15_weights = [0.13333, 0.03333, 0.13333, 0.03333, 0.09539, 0.03333, 0.03333, 0.03333, 0.03707, 0.05699, 0.03897, 0.13333, 0.13333, 0.03825, 0.03333]
msp_epo_30_weights = [0.06667, 0.01667, 0.06667, 0.06667, 0.0387, 0.01667, 0.01667, 0.01667, 0.06667, 0.02469, 0.06667, 0.01667, 0.01667, 0.01667, 0.01667, 0.01667, 0.01667, 0.01667, 0.06667, 0.0443, 0.02974, 0.01667, 0.01667, 0.05967, 0.04331, 0.06667, 0.01667, 0.01667, 0.02625, 0.01667]
msp_epo_45_weights = [0.04444, 0.04379, 0.04444, 0.04444, 0.04444, 0.04444, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.04444, 0.01111, 0.01111, 0.04444, 0.01111, 0.01111, 0.04444, 0.01111, 0.0312, 0.02923, 0.04444, 0.03726, 0.04444, 0.01557, 0.01111, 0.04296, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.04444]

msp_epo_15 = dict(zip(tickers_15, msp_epo_15_weights))
msp_epo_30 = dict(zip(tickers_30, msp_epo_30_weights))
msp_epo_45 = dict(zip(tickers_45, msp_epo_45_weights))

## MAX Returns + EPO weight

In [203]:
max_return_epo_15_weights = [0.13333, 0.03333, 0.13333, 0.03333, 0.13333, 0.03333, 0.03333, 0.03333, 0.03333, 0.03333, 0.03333, 0.13333, 0.13333, 0.03333, 0.03333]
max_return_epo_30_weights = [0.06667, 0.01667, 0.06667, 0.06667, 0.06667, 0.01667, 0.01667, 0.01667, 0.06667, 0.06667, 0.06667, 0.01667, 0.01667, 0.01667, 0.01667, 0.01667, 0.01667, 0.01667, 0.06667, 0.01667, 0.01667, 0.01667, 0.01667, 0.06667, 0.01667, 0.06667, 0.01667, 0.01667, 0.01667, 0.01667]
max_return_epo_45_weights = [0.04444, 0.04444, 0.04444, 0.04444, 0.04444, 0.04444, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.04444, 0.01111, 0.01111, 0.04444, 0.01111, 0.01111, 0.04444, 0.01111, 0.04444, 0.04444, 0.04444, 0.01111, 0.04444, 0.01111, 0.01111, 0.04444, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.04444]

max_return_epo_15 = dict(zip(tickers_15, max_return_epo_15_weights))
max_return_epo_30 = dict(zip(tickers_30, max_return_epo_30_weights))
max_return_epo_45 = dict(zip(tickers_45, max_return_epo_45_weights))

-----

## Rebalncing

In [204]:
# 분기별 (3개월) 리밸런싱
# LookBack period : 1year
# out-of-sample 기간 : 2023-05-01 ~ 2023-10-10

In [205]:
mkt = pd.read_csv('./new_market.csv')
mkt

,pricingDate,A,AAL,AAPL,ABBV,ABT,ACGL,ACN,ADBE,ADI,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
0,2018.5.1,63.574105,42.342797,40.191708,79.940979,53.921780,26.876666,140.357973,224.080000,79.992862,...,180.343060,39.841488,58.326616,47.887170,65.827974,78.555893,108.770822,136.780000,46.462606,80.580399
1,2018.5.2,63.257386,41.249071,41.967178,78.609543,53.032556,26.310000,139.240061,221.100000,79.488744,...,179.572242,39.747543,58.212919,47.839888,64.434146,72.716789,109.245154,133.120000,46.580767,77.198727
2,2018.5.3,63.670080,41.122121,42.043236,78.539055,53.105894,25.983333,140.182433,226.050000,79.137661,...,180.126855,39.696300,58.015844,46.563275,65.135737,74.956321,107.124611,133.540000,45.956201,77.662499
3,2018.5.4,64.303518,41.854527,43.692736,78.452903,53.848442,26.276666,141.762291,228.510000,81.262162,...,180.869471,39.619435,58.288717,47.272504,65.556692,74.738715,108.091876,134.670000,46.808649,78.802606
4,2018.5.7,64.677822,41.541303,44.008850,78.029977,54.380142,26.376666,140.709052,230.990000,81.469211,...,180.399461,39.397382,58.925421,44.369392,65.500564,75.355266,108.808024,137.480000,46.850849,79.372659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1381,2023.10.25,103.400002,11.040000,170.874893,145.259995,93.570000,83.489998,292.679993,521.140015,159.789993,...,88.681633,59.470001,107.606010,31.809999,87.473717,120.309998,103.639999,198.910004,29.273745,163.279068
1382,2023.10.26,104.309998,11.150000,166.670425,145.199997,93.980003,82.449997,292.040008,514.280029,160.860001,...,87.285698,59.770000,106.624985,30.680000,88.400681,118.750000,103.120003,204.830002,30.083887,157.692444
1383,2023.10.27,102.769997,10.920000,167.998672,138.929993,92.849998,81.360001,290.040008,508.119995,160.570007,...,87.026451,58.310001,104.593567,30.600000,87.882370,119.440002,103.190002,207.179993,29.095911,155.657318
1384,2023.10.30,101.169998,11.180000,170.065933,141.889999,93.000000,82.879997,292.700012,526.940002,155.880005,...,87.824135,58.740002,104.920570,29.990000,88.978783,119.870003,103.410004,209.770004,29.619537,156.625000


In [206]:
mkt = pd.read_csv('./new_market.csv')
mkt.set_index('pricingDate', inplace=True)
# mkt.rename_axis('pricingDate', axis='index', inplace=True)
mkt.index = pd.to_datetime(mkt.index)
mkt_in = mkt.loc[:'2023-04-30', :]
mkt_out = mkt.loc['2023-05-01':, :]

In [207]:
# in-sample
mkt_in

,A,AAL,AAPL,ABBV,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
pricingDate,,,,,,,,,,,,,,,,,,,,,
2018-05-01,63.574105,42.342797,40.191708,79.940979,53.921780,26.876666,140.357973,224.08,79.992862,38.801281,...,180.343060,39.841488,58.326616,47.887170,65.827974,78.555893,108.770822,136.78,46.462606,80.580399
2018-05-02,63.257386,41.249071,41.967178,78.609543,53.032556,26.310000,139.240061,221.10,79.488744,38.284275,...,179.572242,39.747543,58.212919,47.839888,64.434146,72.716789,109.245154,133.12,46.580767,77.198727
2018-05-03,63.670080,41.122121,42.043236,78.539055,53.105894,25.983333,140.182433,226.05,79.137661,37.569084,...,180.126855,39.696300,58.015844,46.563275,65.135737,74.956321,107.124611,133.54,45.956201,77.662499
2018-05-04,64.303518,41.854527,43.692736,78.452903,53.848442,26.276666,141.762291,228.51,81.262162,37.793120,...,180.869471,39.619435,58.288717,47.272504,65.556692,74.738715,108.091876,134.67,46.808649,78.802606
2018-05-07,64.677822,41.541303,44.008850,78.029977,54.380142,26.376666,140.709052,230.99,81.469211,37.646635,...,180.399461,39.397382,58.925421,44.369392,65.500564,75.355266,108.808024,137.48,46.850849,79.372659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-24,137.937948,13.320000,164.878984,162.280931,109.873056,72.510000,275.106410,377.34,185.188487,79.896422,...,113.919233,69.814765,116.230360,41.123938,103.855012,137.610165,137.611859,288.19,27.463377,175.963019
2023-04-25,129.859693,12.840000,163.323240,163.091940,109.445108,72.590000,268.607053,369.59,179.423633,75.051218,...,111.083193,69.962199,114.578355,40.657522,102.245627,136.530286,138.528342,284.10,25.967317,172.640326
2023-04-26,132.598928,12.740000,163.313267,160.025930,108.230931,72.290000,269.113109,363.06,178.948181,76.168582,...,108.913871,68.487862,113.526185,40.617827,100.318339,136.728429,138.548265,280.42,25.637019,172.560501


In [208]:
# out-sample
mkt_out

,A,AAL,AAPL,ABBV,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
pricingDate,,,,,,,,,,,,,,,,,,,,,
2023-05-01,135.547342,13.89,169.127363,152.024633,110.579667,75.900000,277.259633,374.150000,180.810368,76.929971,...,115.033747,68.969479,112.759183,42.066692,103.815275,140.909242,138.299221,287.450000,26.054749,179.165975
2023-05-02,133.993448,13.77,168.080227,149.957549,110.967805,76.030000,274.600354,368.660000,181.919755,74.675468,...,112.645503,67.917785,108.265335,41.778904,103.835143,141.573020,142.463240,255.440000,23.237495,176.511812
2023-05-03,134.242470,13.79,166.993201,147.613535,111.226564,75.620000,273.052415,345.250000,180.661789,74.181059,...,108.794459,67.328050,106.131495,40.171259,103.467568,136.034930,139.086200,258.500000,22.013446,177.339990
2023-05-04,133.355954,13.51,165.337729,145.744258,110.330860,72.920000,264.439525,335.830000,179.611833,73.390006,...,108.714851,67.927614,104.272990,40.538437,105.633284,135.054123,136.107632,265.000000,19.361341,178.078366
2023-05-05,132.778224,13.87,173.096506,146.406912,110.728951,74.810000,263.596097,348.400000,181.652313,75.110547,...,110.237357,68.379744,106.868998,40.985005,108.255985,136.213258,137.382738,273.800000,23.082060,185.821338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-25,103.400002,11.04,170.874893,145.259995,93.570000,83.489998,292.679993,521.140015,159.789993,70.612259,...,88.681633,59.470001,107.606010,31.809999,87.473717,120.309998,103.639999,198.910004,29.273745,163.279068
2023-10-26,104.309998,11.15,166.670425,145.199997,93.980003,82.449997,292.040008,514.280029,160.860001,71.407326,...,87.285698,59.770000,106.624985,30.680000,88.400681,118.750000,103.120003,204.830002,30.083887,157.692444
2023-10-27,102.769997,10.92,167.998672,138.929993,92.849998,81.360001,290.040008,508.119995,160.570007,69.608482,...,87.026451,58.310001,104.593567,30.600000,87.882370,119.440002,103.190002,207.179993,29.095911,155.657318


## out-of-sample rebalancing

In [209]:
## market 데이터 불러오기
def downloads(dict, start_date, end_date):
    data = pd.DataFrame()
    a = pd.DataFrame()
    tickers = list(dict.keys())

    for ticker in tqdm(tickers):
        a = mkt.loc[start_date:end_date, ticker]
        data = pd.concat([data, a], axis=1)

    return data

-----------------

## 최종

## out-of-sample rebalancing : Equal Weight

### step01 : market data import

In [210]:
## market 데이터 불러오기
def downloads(dict, start_date, end_date):
    data = pd.DataFrame()
    a = pd.DataFrame()
    tickers = list(dict.keys())

    for ticker in tqdm(tickers):
        a = mkt.loc[start_date:end_date, ticker]
        data = pd.concat([data, a], axis=1)

    return data

In [211]:
type(eq_15_weights)

list

### step02 : 최적화 함수 적용

In [212]:
# Equal Weight
def eq_weight(dict, end, lookback_yr):
    # 종목:가중치 딕셔너리, lookback기간의 마지막 날짜, lookback
    
    # before = pd.to_datetime(end) - pd.DateOffset(years=lookback_yr) + pd.DateOffset(days=1)
    # df = downloads(dict, before, end)  # mkt 데이터에서 일별 주가데이터 가져오기(lookback 기간만큼)

    if len(dict) == 15:
        return eq_15_weights
    if len(dict) == 30:
        return eq_30_weights
    if len(dict) == 45:
        return eq_45_weights
    else:
        print('dict를 다시 입력하세요')   

### step03 : rebalancing

In [213]:
# start부터 3개월간 투자 시, 수익률과 (손)수익 확인하는 함수
# parameter: 시작시점, 끝시점, 종목 딕서녀리, 최적 종목 들어있는 리스트, lookback 기간(int)
def my_return04(start, end, stock_dict, ticker_lst, lookback):

    ret_rebal = []  # 리밸런싱 시점에서의 수익률(소수점 단위)
    money_rebal = []  # 리밸런싱 시점에서의 (손)수익
    initial = 10000  # 초기 투자 금액 (1만 달러)
    w = []

      # 시작 시점 기준 3개월 뒤 시점
    dates = pd.date_range(start, end, freq='3MS')
    # ['2023-05-01', '2023-08-01']

    for i in range(len(dates)):
        start = dates[i]
        rebal = start + pd.DateOffset(months=3) - pd.DateOffset(days=1)
        # w* 이용하여 start 시점부터 3개월간 투자할 때, 3개월 뒤의 종가 확인
        price_3m = mkt_out.loc[rebal:, ticker_lst][0:1]  # 3개월 뒤 시점의 종가 df
        lst_3m = [] # 3개월 뒤 시점의 종가 저장 리스트
        for i in price_3m.columns:
            lst_3m.append(float(price_3m[i]))
            # lst_3m.append(price_3m[i])

        # w* 이용하여 start 시점 당시의 초기 투자량 확인
        price_now = mkt_out.loc[start:, ticker_lst][0:1]  # start 시점의 종가 df
        lst_now = []  # start 시점의 종가 저장 리스트
        for j in price_now.columns:
            lst_now.append(float(price_now[j]))
            # lst_now.append(price_now[j])

        # 리밸런싱을 위한 GMV + EPO 최적화
        opt_w = list(eq_weight(stock_dict, start, lookback))  # 리밸런싱 시점에서의 최적 가중치 w**
        w.append(opt_w)

        # 초기 투자 주식 보유량 (XX주)
        a = np.multiply(opt_w, ([initial] * len(ticker_lst) ) ) 
        count_stocks = np.divide(a, np.array(lst_now))

        # 3개월 동안의 투자 수익률 확인 (소수점 단위) : (3개월 뒤 시점의 payoff - start 시점의 payoff) / start 시점의 payoff
        b = sum(np.multiply(count_stocks, np.array(lst_3m)))
        ret_3m = (b - initial) / initial
        ret_rebal.append(ret_3m)

        # 3개월 후 총 투자 (손)수익 (달러)
        total_ret_3m = initial * (1 + ret_3m)
        initial = total_ret_3m
        money_rebal.append(total_ret_3m)

    return ret_rebal, money_rebal, w

### step04 : results <lookback 1yr>
- 리밸런싱 시점에서의 총 (손)수익, 리밸런싱 시점에서의 (손)수익률(소수점 단위)
- 마지막 시점에서의 총 (손)수익, 마지막 시점에서의 (손)수익률(소수점 단위)

In [214]:
# 15종목
ret, money, w = my_return04('2023-05-01', '2023-10-01', stocks_15, tickers_15, 1)
ret, money, w
print('-------------------------------------------------------')
print('리밸런싱 시점에서의 (손)수익률(소수점 단위) :', ret[0])
print('리밸런싱 시점에서의 (손)수익 : $', money[0])
print()
print('마지막 시점에서의 총 (손)수익률(소수점 단위) :', ret[1])
print('마지막 시점에서의 총 (손)수익 : $', money[1])

-------------------------------------------------------
리밸런싱 시점에서의 (손)수익률(소수점 단위) : 0.13548976794650133
리밸런싱 시점에서의 (손)수익 : $ 11354.897679465012

마지막 시점에서의 총 (손)수익률(소수점 단위) : -0.09617769762152527
마지막 시점에서의 총 (손)수익 : $ 10262.809763926067


C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_8216\259674884.py:21: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_3m.append(float(price_3m[i]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_8216\259674884.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_now.append(float(price_now[j]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_8216\259674884.py:21: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_3m.append(float(price_3m[i]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_8216\259674884.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_now.append(float(price_now[j

In [215]:
# 30종목
ret, money, w = my_return04('2023-05-01', '2023-10-01', stocks_30, tickers_30, 1)
ret, money, w
print('-------------------------------------------------------')
print('리밸런싱 시점에서의 (손)수익률(소수점 단위) :', ret[0])
print('리밸런싱 시점에서의 (손)수익 : $', money[0])
print()
print('마지막 시점에서의 총 (손)수익률(소수점 단위) :', ret[1])
print('마지막 시점에서의 총 (손)수익 : $', money[1])

-------------------------------------------------------
리밸런싱 시점에서의 (손)수익률(소수점 단위) : 0.11093751512478284
리밸런싱 시점에서의 (손)수익 : $ 11109.375151247828

마지막 시점에서의 총 (손)수익률(소수점 단위) : -0.08379379158264479
마지막 시점에서의 총 (손)수익 : $ 10178.478485210755


C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_8216\259674884.py:21: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_3m.append(float(price_3m[i]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_8216\259674884.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_now.append(float(price_now[j]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_8216\259674884.py:21: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_3m.append(float(price_3m[i]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_8216\259674884.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_now.append(float(price_now[j

In [216]:
# 45종목
ret, money, w = my_return04('2023-05-01', '2023-10-01', stocks_45, tickers_45, 1)
ret, money, w
print('-------------------------------------------------------')
print('리밸런싱 시점에서의 (손)수익률(소수점 단위) :', ret[0])
print('리밸런싱 시점에서의 (손)수익 : $', money[0])
print()
print('마지막 시점에서의 총 (손)수익률(소수점 단위) :', ret[1])
print('마지막 시점에서의 총 (손)수익 : $', money[1])

-------------------------------------------------------
리밸런싱 시점에서의 (손)수익률(소수점 단위) : 0.09017982254978851
리밸런싱 시점에서의 (손)수익 : $ 10901.798225497885

마지막 시점에서의 총 (손)수익률(소수점 단위) : -0.07889269023119004
마지막 시점에서의 총 (손)수익 : $ 10041.726035130743


C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_8216\259674884.py:21: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_3m.append(float(price_3m[i]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_8216\259674884.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_now.append(float(price_now[j]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_8216\259674884.py:21: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_3m.append(float(price_3m[i]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_8216\259674884.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_now.append(float(price_now[j

### step04 : results <lookback 2yr>
- 리밸런싱 시점에서의 총 (손)수익, 리밸런싱 시점에서의 (손)수익률(소수점 단위)
- 마지막 시점에서의 총 (손)수익, 마지막 시점에서의 (손)수익률(소수점 단위)

In [217]:
# 15종목
ret, money = my_return04('2023-05-01', '2023-10-01', stocks_15, tickers_15, 2)
ret, money
print('-------------------------------------------------------')
print('리밸런싱 시점에서의 (손)수익률(소수점 단위) :', ret[0])
print('리밸런싱 시점에서의 (손)수익 : $', money[0])
print()
print('마지막 시점에서의 총 (손)수익률(소수점 단위) :', ret[1])
print('마지막 시점에서의 총 (손)수익 : $', money[1])

C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_8216\259674884.py:21: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_3m.append(float(price_3m[i]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_8216\259674884.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_now.append(float(price_now[j]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_8216\259674884.py:21: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_3m.append(float(price_3m[i]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_8216\259674884.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_now.append(float(price_now[j

ValueError: too many values to unpack (expected 2)

In [ ]:
# 30종목
ret, money = my_return04('2023-05-01', '2023-10-01', stocks_30, tickers_30, 2)
ret, money
print('-------------------------------------------------------')
print('리밸런싱 시점에서의 (손)수익률(소수점 단위) :', ret[0])
print('리밸런싱 시점에서의 (손)수익 : $', money[0])
print()
print('마지막 시점에서의 총 (손)수익률(소수점 단위) :', ret[1])
print('마지막 시점에서의 총 (손)수익 : $', money[1])

C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_4716\2063037324.py:20: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_3m.append(float(price_3m[i]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_4716\2063037324.py:26: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_now.append(float(price_now[j]))
100%|██████████| 30/30 [00:00<00:00, 999.76it/s]
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_4716\2063037324.py:20: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_3m.append(float(price_3m[i]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_4716\2063037324.py:26: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(se

stock_dict를 다시 입력하세요


100%|██████████| 30/30 [00:00<00:00, 999.78it/s]

stock_dict를 다시 입력하세요
-------------------------------------------------------
리밸런싱 시점에서의 (손)수익률(소수점 단위) : 0.10567674326320048
리밸런싱 시점에서의 (손)수익 : $ 11056.767432632005

마지막 시점에서의 총 (손)수익률(소수점 단위) : -0.08765533362810467
마지막 시점에서의 총 (손)수익 : $ 10087.582794476284


In [ ]:
# 45종목
ret, money = my_return04('2023-05-01', '2023-10-01', stocks_45, tickers_45, 2)
ret, money
print('-------------------------------------------------------')
print('리밸런싱 시점에서의 (손)수익률(소수점 단위) :', ret[0])
print('리밸런싱 시점에서의 (손)수익 : $', money[0])
print()
print('마지막 시점에서의 총 (손)수익률(소수점 단위) :', ret[1])
print('마지막 시점에서의 총 (손)수익 : $', money[1])

C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_4716\2063037324.py:20: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_3m.append(float(price_3m[i]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_4716\2063037324.py:26: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_now.append(float(price_now[j]))
100%|██████████| 45/45 [00:00<00:00, 1022.50it/s]
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_4716\2063037324.py:20: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lst_3m.append(float(price_3m[i]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_4716\2063037324.py:26: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(s

-------------------------------------------------------
리밸런싱 시점에서의 (손)수익률(소수점 단위) : 0.08418375846069048
리밸런싱 시점에서의 (손)수익 : $ 10841.837584606907

마지막 시점에서의 총 (손)수익률(소수점 단위) : -0.08363198148645226
마지막 시점에서의 총 (손)수익 : $ 9935.11322445194


## in-sample rebalancing : Equal Weight

### step01 : market data import

In [ ]:
## market 데이터 불러오기
def downloads(dict, start_date, end_date):
    data = pd.DataFrame()
    a = pd.DataFrame()
    tickers = list(dict.keys())

    for ticker in tqdm(tickers):
        a = mkt.loc[start_date:end_date, ticker]
        data = pd.concat([data, a], axis=1)

    return data

### step02 : 최적화 함수 적용

In [ ]:
# Equal Weight
def eq_weight(stock_dict, end, lookback_yr):
    # 종목:가중치 딕셔너리, lookback기간의 마지막 날짜, lookback
    
    # before = pd.to_datetime(end) - pd.DateOffset(years=lookback_yr) + pd.DateOffset(days=1)
    # df = downloads(stock_dict, before, end)  # mkt 데이터에서 일별 주가데이터 가져오기(lookback 기간만큼)

    if len(stock_dict) == 15:
        opt_w = [1/15] * 15
    elif len(stock_dict) == 30:
        opt_w = [1/30] * 30
    elif len(stock_dict) == 45:
        opt_w = [1/45] * 45
    else:
        print('stock_dict를 다시 입력하세요')   

    return opt_w

### step03 : rebalancing

In [ ]:
# start부터 3개월간 투자 시, 수익률과 (손)수익 확인하는 함수
# parameter: 시작시점, 끝시점, 종목 딕서녀리, 최적 종목 들어있는 리스트, lookback 기간(int)
def my_return04(start, end, stock_dict, ticker_lst, lookback):

    ret_rebal = []  # 리밸런싱 시점에서의 수익률(소수점 단위)
    money_rebal = []  # 리밸런싱 시점에서의 (손)수익
    initial = 10000  # 초기 투자 금액 (1만 달러)

      # 시작 시점 기준 3개월 뒤 시점
    dates = pd.date_range(start, end, freq='3MS')
    # 2018-05-01부터 2023-04-30까지 3개월 간격으로 리스트 요소 존재

    for i in range(len(dates)):
        start = dates[i]
        rebal = start + pd.DateOffset(months=3) - pd.DateOffset(days=1)
        # w* 이용하여 start 시점부터 3개월간 투자할 때, 3개월 뒤의 종가 확인
        price_3m = mkt_in.loc[rebal:, ticker_lst][0:1]  # 3개월 뒤 시점의 종가 df
        lst_3m = [] # 3개월 뒤 시점의 종가 저장 리스트
        for i in price_3m.columns:
            lst_3m.append(float(price_3m[i].iloc[0]))

        # w* 이용하여 start 시점 당시의 초기 투자량 확인
        price_now = mkt_in.loc[start:, ticker_lst][0:1]  # start 시점의 종가 df
        lst_now = []  # start 시점의 종가 저장 리스트
        for j in price_now.columns:
            lst_now.append(float(price_now[j].iloc[0]))

        # 리밸런싱을 위한 GMV + EPO 최적화
        opt_w = list(eq_weight(stock_dict, start, lookback))  # 리밸런싱 시점에서의 최적 가중치 w**

        # 초기 투자 주식 보유량 (XX주)
        a = np.multiply(opt_w, ([initial] * len(ticker_lst) ) ) 
        count_stocks = np.divide(a, np.array(lst_now))

        # 3개월 동안의 투자 수익률 확인 (소수점 단위) : (3개월 뒤 시점의 payoff - start 시점의 payoff) / start 시점의 payoff
        b = sum(np.multiply(count_stocks, np.array(lst_3m)))
        ret_3m = (b - initial) / initial
        ret_rebal.append(ret_3m)

        # 3개월 후 총 투자 (손)수익 (달러)
        total_ret_3m = initial * (1 + ret_3m)
        initial = total_ret_3m
        money_rebal.append(total_ret_3m)

    return ret_rebal, money_rebal

In [ ]:
# a = np.array([[1,2,3,4,5],[1,2,3,4,5]])
# b = np.array([[2,3,4,5,6],[1,2,3,4,5]])
# np.divide(a,b)

array([[0.5       , 0.66666667, 0.75      , 0.8       , 0.83333333],
       [1.        , 1.        , 1.        , 1.        , 1.        ]])

In [ ]:
# start부터 3개월간 투자 시, 수익률과 (손)수익 확인하는 함수
# parameter: 시작시점, 끝시점, 종목 딕서녀리, 최적 종목 들어있는 리스트, lookback 기간(int)
def my_return04(start, end, stock_dict, ticker_lst, lookback):

    ret_rebal = []  # 리밸런싱 시점에서의 수익률(소수점 단위)
    money_rebal = []  # 리밸런싱 시점에서의 (손)수익
    initial = 10000  # 초기 투자 금액 (1만 달러)
    df_rebals = pd.DataFrame()  # 리밸런싱 시점의 종가를 저장할 데이터프레임

      # 시작 시점 기준 3개월 뒤 시점
    dates = pd.date_range(start, end, freq='3MS')
    # 2018-05-01부터 2023-04-30까지 3개월 간격으로 리스트 요소 존재

    for i in range(len(dates)):
        start = dates[i]
        rebal = start + pd.DateOffset(months=3) - pd.DateOffset(days=1)
        # 3개월 뒤 시점의 종가 확인
        price_3m = mkt_in.loc[:rebal, ticker_lst][-1:]  # 3개월 뒤 시점의 종가 df

        # df_rebals 데이터프레임에 해당 종가 추가
        #df_rebals = pd.concat([df_rebals, price_3m])

        # w* 이용하여 start 시점 당시의 초기 투자량 확인
        price_now = mkt_in.loc[start:, ticker_lst][0:1]  # start 시점의 종가 df
        lst_now = []  # start 시점의 종가 저장 리스트
        for j in price_now.columns:
            lst_now.append(float(price_now[j].iloc[0]))

        # 리밸런싱을 위한 GMV + EPO 최적화
        opt_w = list(eq_weight(stock_dict, start, lookback))  # 리밸런싱 시점에서의 최적 가중치 w**

        # 초기 투자 주식 보유량 (XX주)
        a = np.multiply(opt_w, ([initial] * len(ticker_lst) ) ) 
        count_stocks = np.divide(a, np.array(lst_now))

        # 리밸런싱 시점에서의 투자 수익률 확인 (소수점 단위) : (리밸런싱 시점의 payoff - start 시점의 payoff) / start 시점의 payoff
        b = sum(np.multiply(count_stocks, np.array(price_3m)))
        ret_3m = (b - initial) / initial
        ret_rebal.append(ret_3m)

        # 리밸런싱 시점에서의 총 투자 (손)수익 (달러)
        total_ret_3m = initial * (1 + ret_3m)
        initial = total_ret_3m
        money_rebal.append(total_ret_3m)

    return ret_rebal, money_rebal

In [ ]:
# opt_w = [1/15] * 15
# lst_now = []
# price_now = mkt_in.loc['2018-05-01':, tickers_15][0:1]  # start 시점의 종가 df
# for j in price_now.columns:
#     lst_now.append(float(price_now[j].iloc[0]))

# # 초기 투자 주식 보유량 (XX주)
# a = np.multiply(opt_w, ([initial] * 15 ) ) 
# b = np.array(lst_now).shape
# #count_stocks = np.divide(a, np.array(lst_now))
# np.divide(a,b)


array([44.44444444, 44.44444444, 44.44444444, 44.44444444, 44.44444444,
       44.44444444, 44.44444444, 44.44444444, 44.44444444, 44.44444444,
       44.44444444, 44.44444444, 44.44444444, 44.44444444, 44.44444444])

### step04 : results <lookback 1yr>

In [ ]:
# 15종목
ret, money = my_return04('2018-05-01', '2023-04-30', stocks_15, tickers_15, 1)
ret = pd.DataFrame(ret)
money = pd.DataFrame(money)
money['Total Return'] = money.sum(axis=1)
ret['최종 손수익률'] = ret.mean(axis = 1)
print('최종 손수익 : $', money.loc[19, 'Total Return'])
print('최종 누적 손수익률 : %', ((money.loc[19, 'Total Return'] - initial) / initial)*100)

최종 손수익 : $ 22127.321673091534
최종 누적 손수익률 : % 121.27321673091534


In [ ]:
# 30종목
ret, money = my_return04('2018-05-01', '2023-04-30', stocks_30, tickers_30, 1)
ret = pd.DataFrame(ret)
money = pd.DataFrame(money)
money['Total Return'] = money.sum(axis=1)
ret['최종 손수익률'] = ret.mean(axis = 1)
print('최종 손수익 : $', money.loc[19, 'Total Return'])
print('최종 누적 손수익률 : %', ((money.loc[19, 'Total Return'] - initial) / initial)*100)

최종 손수익 : $ 18627.190010459963
최종 누적 손수익률 : % 86.27190010459962


In [ ]:
# 45종목
ret, money = my_return04('2018-05-01', '2023-04-30', stocks_45, tickers_45, 1)
ret = pd.DataFrame(ret)
money = pd.DataFrame(money)
money['Total Return'] = money.sum(axis=1)
ret['최종 손수익률'] = ret.mean(axis = 1)
print('최종 손수익 : $', money.loc[19, 'Total Return'])
print('최종 누적 손수익률 : %', ((money.loc[19, 'Total Return'] - initial) / initial)*100)

최종 손수익 : $ 17957.052718191528
최종 누적 손수익률 : % 79.57052718191527


### step04 : results <lookback 2yr>

In [ ]:
# 15종목
ret, money = my_return04('2018-05-01', '2023-04-30', stocks_15, tickers_15, 2)
ret = pd.DataFrame(ret)
money = pd.DataFrame(money)
money['Total Return'] = money.sum(axis=1)
ret['최종 손수익률'] = ret.mean(axis = 1)
print('최종 손수익 : $', money.loc[19, 'Total Return'])
print('최종 누적 손수익률 : %', ((money.loc[19, 'Total Return'] - initial) / initial)*100)

최종 손수익 : $ 22127.321673091534
최종 누적 손수익률 : % 121.27321673091534


In [ ]:
# 30종목
ret, money = my_return04('2018-05-01', '2023-04-30', stocks_30, tickers_30, 2)
ret = pd.DataFrame(ret)
money = pd.DataFrame(money)
money['Total Return'] = money.sum(axis=1)
ret['최종 손수익률'] = ret.mean(axis = 1)
print('최종 손수익 : $', money.loc[19, 'Total Return'])
print('최종 누적 손수익률 : %', ((money.loc[19, 'Total Return'] - initial) / initial)*100)

최종 손수익 : $ 18627.190010459963
최종 누적 손수익률 : % 86.27190010459962


In [ ]:
# 45종목
ret, money = my_return04('2018-05-01', '2023-04-30', stocks_45, tickers_45, 2)
ret = pd.DataFrame(ret)
money = pd.DataFrame(money)
money['Total Return'] = money.sum(axis=1)
ret['최종 손수익률'] = ret.mean(axis = 1)
print('최종 손수익 : $', money.loc[19, 'Total Return'])
print('최종 누적 손수익률 : %', ((money.loc[19, 'Total Return'] - initial) / initial)*100)

최종 손수익 : $ 17957.052718191528
최종 누적 손수익률 : % 79.57052718191527
